In [3]:
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import tiktoken

#special token ids
enc = tiktoken.get_encoding('cl100k_base')
SOS_TOKEN_ID = enc.encode("<|startoftext|>", allowed_special="all")[0]
EOS_TOKEN_ID = enc.encode("<|endoftext|>", allowed_special="all")[0]
PAD_TOKEN_ID = 0

# HP
batch_size = 32
block_size = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
vocab_size = 100_000
n_embed = 256
n_layers = 6
learning_rate = 1e-3
max_iters = 10000
eval_interval = 100
